<a href="https://colab.research.google.com/github/allenphos/ML-course/blob/main/Copy_of_HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Download the dataset
raw_df = pd.read_csv('drive/MyDrive/Colab Notebooks/data/customer_segmentation_train.csv')

raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [12]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [13]:
raw_df.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [14]:
raw_df.isna().sum()

,0
ID,0
Gender,0
Ever_Married,140
Age,0
Graduated,78
Profession,124
Work_Experience,829
Spending_Score,0
Family_Size,335
Var_1,76


In [15]:
# Create inputs and targets
input_cols = raw_df.drop(columns=['Segmentation'])
target_col = raw_df['Segmentation']

# Identify numeric and categorical columns
numeric_cols = input_cols.select_dtypes(include=np.number).columns.tolist()
categorical_cols = input_cols.select_dtypes('object').columns.tolist()

In [16]:
null_count = input_cols.isnull().sum()
null_percentage = round((input_cols.isnull().sum() / input_cols.shape[0]) * 100, 2)

# Creating a DataFrame to Summarize Missing Data
null_df = pd.DataFrame({
    'column_name': input_cols.columns,
    'null_count': null_count,
    'null_percentage': null_percentage
})

null_df.reset_index(drop=True, inplace=True)  # Resetting the Index

# Sorting by null_percentage in descending order
null_df_sorted = null_df.sort_values(by='null_percentage', ascending=False)

null_df_sorted

,column_name,null_count,null_percentage
6,Work_Experience,829,10.28
8,Family_Size,335,4.15
2,Ever_Married,140,1.74
5,Profession,124,1.54
4,Graduated,78,0.97
9,Var_1,76,0.94
0,ID,0,0.00
1,Gender,0,0.00
3,Age,0,0.00
7,Spending_Score,0,0.00


In [17]:
target_col.value_counts()

,count
Segmentation,
D,2268
A,1972
C,1970
B,1858


In [47]:
# Create preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# inputs_train inputs_test targets_train targets_test
X_train_raw, X_val_raw, y_train, y_val = train_test_split(input_cols, target_col, test_size=0.2,stratify=target_col, random_state=42)

# Fit and transform the training data
X_train = preprocessor.fit_transform(X_train_raw)

# Transform the validation data (based on the training fit)
X_val = preprocessor.transform(X_val_raw)


In [19]:
# Displaying the transformed datasets
pd.set_option('display.max_columns', None)
display(pd.DataFrame(X_train).head()), display(pd.DataFrame(X_val).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,0.931296,-0.695320,1.942754,-1.227022,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.220807,1.703982,0.000000,-0.560068,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.697318,-0.635337,-0.513120,0.773838,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.051676,0.264401,-0.820105,2.107745,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-1.389029,-0.935250,1.942754,-1.227022,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,-0.341697,-0.155477,2.863707,0.106885,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-1.424336,-1.355128,0.407832,0.773838,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.001286,-0.455390,-0.820105,-1.227022,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.965069,-0.995233,-0.820105,-0.560068,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-1.712554,-1.535076,-0.820105,2.107745,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


(None, None)

In [20]:
print("Розміри:", X_train.shape, y_train.shape)


Розміри: (6454, 33) (6454,)


In [21]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))  # Перевірка розподілу класів

{'A': 1578, 'B': 1486, 'C': 1576, 'D': 1814}


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [22]:
from imblearn.over_sampling import SMOTENC

# Визначаємо індекси категоріальних ознак
cat_feature_indices = list(range(len(numeric_cols), X_train.shape[1]))

# Використовуємо SMOTENC (з підтримкою категоріальних ознак)
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

print("Розміри після SMOTENC:", X_train_smotenc.shape, y_train_smotenc.shape)

Розміри після SMOTENC: (7256, 33) (7256,)


In [23]:
unique, counts = np.unique(y_train_smotenc, return_counts=True)
print(dict(zip(unique, counts)))  # Перевірка розподілу класів

{'A': 1814, 'B': 1814, 'C': 1814, 'D': 1814}


In [24]:
from imblearn.combine import SMOTETomek

# Використовуємо SMOTE-Tomek для подальшого балансування
smote_tomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smote_tomek.fit_resample(X_train_smotenc, y_train_smotenc)

print("Розміри після SMOTE-Tomek:", X_train_smotetomek.shape, y_train_smotetomek.shape)


Розміри після SMOTE-Tomek: (5712, 33) (5712,)


In [25]:
unique, counts = np.unique(y_train_smotetomek, return_counts=True)
print(dict(zip(unique, counts)))  # Перевірка розподілу класів

{'A': 1408, 'B': 1398, 'C': 1415, 'D': 1491}


 SMOTENC додає синтетичні зразки для числових і категоріальних ознак. збільшує кількість прикладів у меншості (oversampling).

SMOTE-Tomek додатково чистить дані, видаляючи дублювання та погано розташовані зразки. поєднує SMOTE (oversampling) і Tomek Links (undersampling) для кращого балансу класів.

Апсемплінг (Upsampling)
Апсемплінг – це збільшення кількості зразків для менш представленого класу.

✅ Методи апсемплінгу:

SMOTE (Synthetic Minority Over-sampling Technique) – генерує нові синтетичні зразки.
Random Oversampling – просто дублює наявні приклади меншого класу.

SMOTENC – спеціальна версія SMOTE для категоріальних даних.

Ресемплінг (Resampling)
Ресемплінг – будь-яка зміна кількості зразків (зменшення або збільшення).
🔹 Використовується, щоб отримати більш збалансований набір даних.

✅ Методи ресемплінгу:

SMOTE + Tomek Links (SMOTE-Tomek) – додає синтетичні зразки (SMOTE), а потім видаляє проблемні пари з Tomek Links.

Random Undersampling – випадково видаляє зразки більшості.

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

## Original data

In [26]:
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

# Логістична регресія зі стратегією one-vs-rest (OvR)
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, y_train)
ovr_predictions = ovr_model.predict(X_val)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_val, ovr_predictions))

              precision    recall  f1-score   support

           A       0.43      0.47      0.45       394
           B       0.41      0.17      0.24       372
           C       0.49      0.62      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.52      0.49      1614



In [27]:
ovr_model.classes_

array(['A', 'B', 'C', 'D'], dtype='<U1')

In [28]:
roc_auc = roc_auc_score(y_val, ovr_model.predict_proba(X_val), multi_class='ovr')
print(f"Multi-class ROC AUC: {roc_auc:.4f}")


Multi-class ROC AUC: 0.7688


## SMOTENC

In [29]:
# Логістична регресія зі стратегією one-vs-rest (OvR)
ovr_model_smnc = OneVsRestClassifier(log_reg)
ovr_model_smnc.fit(X_train_smotenc, y_train_smotenc)
ovr_predictions_smnc = ovr_model_smnc.predict(X_val)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_val, ovr_predictions_smnc))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.39      0.23      0.29       372
           C       0.50      0.59      0.54       394
           D       0.66      0.72      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



In [30]:
roc_auc = roc_auc_score(y_val, ovr_model_smnc.predict_proba(X_val), multi_class='ovr')
print(f"Multi-class ROC AUC: {roc_auc:.4f}")


Multi-class ROC AUC: 0.7672


## SMOTE-Tomek

In [31]:
# Логістична регресія зі стратегією one-vs-rest (OvR)
ovr_model_smt = OneVsRestClassifier(log_reg)
ovr_model_smt.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions_smt = ovr_model_smt.predict(X_val)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_val, ovr_predictions_smt))

              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.39      0.25      0.30       372
           C       0.51      0.59      0.55       394
           D       0.67      0.71      0.69       454

    accuracy                           0.51      1614
   macro avg       0.50      0.50      0.50      1614
weighted avg       0.50      0.51      0.50      1614



In [32]:
roc_auc = roc_auc_score(y_val, ovr_model_smt.predict_proba(X_val), multi_class='ovr')
print(f"Multi-class ROC AUC: {roc_auc:.4f}")

Multi-class ROC AUC: 0.7680


Так як задача багатокласова, потрібно використовувати roc_auc_score з multi_class='ovr'.

- На оригінальних даних з OvR ROC AUC: 0.7688
- SMOTENC Multi-class ROC AUC:         0.7672
- SMOTE-Tomek Multi-class ROC AUC:     0.7680

Дані без збалансовання показують найкращій результат.

Якщо дивитись на f1-score, то клас D має найкращу якість передбачення 70%, потім С - 55%, А - 44% та найменша якість у B класу - 24-30%. Що згоджується з розподіленням класів в оригінальних даних.

По метрикам macro avg та weighted avg крайщій результат показує модель зі збалансуванням SMOTE-Tomek - 50%. Тобто балансування трохи покращило точність на менших по кількості даних класів, але не збільшило загальну точність моделі.

**Висновок**: Группа D більше всього підходить для класифікування нових клієнтів.


classification_report показує якість класифікації за різними метриками:

- Precision (точність) – скільки передбачених позитивних класів виявилися правильними.
- Recall (повнота) – скільки справжніх позитивних класів ми знайшли. Наскільки добре модель виявила цей клас.
- F1-score – середнє значення precision та recall. Середнє значення precision і recall (важливе, якщо дані незбалансовані).
- Support – кількість прикладів для кожного класу у тестовому наборі.

Accuracy (Точність всієї моделі) → 51% правильних передбачень.

Macro avg (Середнє між усіма класами) → Дає рівне значення для всіх класів.

Weighted avg (Зважене середнє) → Бере до уваги кількість прикладів у кожному класі.
